# HW3

## Text Processing

### Q1

1. Modify the code I wrote in lecture 8 with what you have learnt in lecture 9 and correctly tokenize the text both on the word and sentence level, and by removing the stopwords. Rewrite the `getSummary` function and all the other functions that it depends by maing these corrections.

2. Rewrite the code I wrote for `getKeywords` function making the same corrections.

3. Test your code from parts 1 and 2 on random articles from the Guardian.

4. Rewrite the `getSubjectGuardian` function for another newspaper in English, and test your code from part 1 and 2 on random articles from this new newspaper.

# Answers
## Q1


First we define the libraries.

In [1]:
import requests
import nltk
import regex as re
import numpy as np
import spacy
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from snowballstemmer import TurkishStemmer
from bs4 import BeautifulSoup
from collections import Counter
from xmltodict import parse
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA

## Data Scraping

We edit xml type data by pulling.

In [2]:
link=requests.get(f'https://www.milliyet.com.tr/rss/rssNew/dunyaRss.xml')
raw = parse(link.text)
raw

OrderedDict([('rss',
              OrderedDict([('@xmlns:atom', 'http://www.w3.org/2005/Atom'),
                           ('@version', '2.0'),
                           ('channel',
                            OrderedDict([('title', 'Milliyet'),
                                         ('link',
                                          'https://www.milliyet.com.tr'),
                                         ('description', 'Son Haberler'),
                                         ('language', 'tr'),
                                         ('managingEditor',
                                          'internet@milliyet.com.tr'),
                                         ('category', 'dunya'),
                                         ('image',
                                          OrderedDict([('title', 'Milliyet'),
                                                       ('link',
                                                        'https://www.milliyet.com.tr'),
                  

I dealt with the first news in the economics section of Milliyet newspaper.

In [3]:
news=raw['rss']['channel']['item']
url=news[0]['atom:link']['@href']
url

'https://www.milliyet.com.tr/dunya/kanada-ukraynadaki-buyukelciligini-acmayi-planliyor-6747100'

In [4]:
def getText(url):
    link=requests.get(url)
    raw = BeautifulSoup(link.content,'html.parser')
    return ' '.join([x.text for x in raw.find_all('p')])

The final version of the text I edited with BeautifulSoup:

In [7]:
omats=getText(url)
omats

'Rusya-Ukrayna savaşı devam ederken, Kanada’nın Ukrayna’daki büyükelçiliğini kısa süre içinde yeniden açmayı planladığı açıklandı. Kanada Dışişleri Bakanı Melanie Joly yaptığı açıklamada, “Hedefimiz önümüzdeki günlerde veya haftalarda bunu gerçekleştirmek. Sadece personel için güvenli bir ortam olduğundan emin olmamız gerekiyor. Ayrıca diğer meslektaşlarımızın ve müttefiklerimizin ne yaptığına bakıyoruz” dedi. Kanada\'nın eski Rusya büyükelçisi Jeremy Kinsman ise, Kanada\'nın diplomatik varlığını Ukrayna\'dan asla taşımaması gerektiğini belirterek, "Asla ayrılmamalıydık. Büyükelçiliği kapatmamalıydık. Hükümetlerin geçmişten öğrendiklerimizi neden tekrar tekrar unuttuğunu bilmiyorum. Büyükelçiliklerimiz bir hükümete akreditedir ve siz hükümetten ayrılmıyorsunuz" ifadelerini kullandı Kanada, geçtiğimiz 12 Şubat\'ta Ukrayna\'nın başkenti Kiev\'deki büyükelçiliğini kapatmış ve Ukrayna\'da Rus saldırıları tehditlerinin artması üzerine diplomatik personelini Ukrayna\'nın Lviv şehrine taşımış

We use the following function to find the number of all words and sentences in the text.

In [8]:
OMATS={'sentences': sent_tokenize(omats),
         'words': word_tokenize(omats)}
(len(OMATS['sentences']),len(OMATS['words']))

(10, 180)

We clean up the phrases and words we find.

In [9]:
swTR = set(stopwords.words('turkish'))
OMATS.update({'cleanedSentences': [re.sub(r'[^\p{L}\s]','',sentence.lower()) for sentence in OMATS['sentences']],
              'cleanedWords': [re.sub(r'[^\p{L}]','',word.lower()) for word in OMATS['words']]})
OMATS['cleanedWords']

['rusyaukrayna',
 'savaşı',
 'devam',
 'ederken',
 '',
 'kanada',
 '',
 'nın',
 'ukrayna',
 '',
 'daki',
 'büyükelçiliğini',
 'kısa',
 'süre',
 'içinde',
 'yeniden',
 'açmayı',
 'planladığı',
 'açıklandı',
 '',
 'kanada',
 'dışişleri',
 'bakanı',
 'melanie',
 'joly',
 'yaptığı',
 'açıklamada',
 '',
 '',
 'hedefimiz',
 'önümüzdeki',
 'günlerde',
 'veya',
 'haftalarda',
 'bunu',
 'gerçekleştirmek',
 '',
 'sadece',
 'personel',
 'için',
 'güvenli',
 'bir',
 'ortam',
 'olduğundan',
 'emin',
 'olmamız',
 'gerekiyor',
 '',
 'ayrıca',
 'diğer',
 'meslektaşlarımızın',
 've',
 'müttefiklerimizin',
 'ne',
 'yaptığına',
 'bakıyoruz',
 '',
 'dedi',
 '',
 'kanadanın',
 'eski',
 'rusya',
 'büyükelçisi',
 'jeremy',
 'kinsman',
 'ise',
 '',
 'kanadanın',
 'diplomatik',
 'varlığını',
 'ukraynadan',
 'asla',
 'taşımaması',
 'gerektiğini',
 'belirterek',
 '',
 '',
 'asla',
 'ayrılmamalıydık',
 '',
 'büyükelçiliği',
 'kapatmamalıydık',
 '',
 'hükümetlerin',
 'geçmişten',
 'öğrendiklerimizi',
 'neden',
 't

In [10]:
def getSubjectMilliyet(subject):
    link=requests.get(f'https://www.milliyet.com.tr/rss/rssNew/{subject}Rss.xml')
    raw = parse(link.text)
    return raw['rss']['channel']['item']

In [11]:
def processText(text):
    swTR = set(stopwords.words('turkish'))
    OMATS.update({'cleanedSentences': [re.sub(r'[^\p{L}\s]','',sentence.lower()) for sentence in OMATS['sentences']],
              'cleanedWords': [re.sub(r'[^\p{L}]','',word.lower()) for word in OMATS['words']]})
    return OMATS

We convert the cleaned text into a matrix (word, sentence).

In [12]:
def countPieces(text):
    words = OMATS['cleanedWords']
    sentences = OMATS['cleanedSentences']
    return (len(words),len(sentences))

In [ ]:
def getMatrix(sentences):
    vectorizer = CountVectorizer()
    return vectorizer.fit_transform(sentences)

In [13]:
countPieces(getText(url))

(180, 10)

In [14]:
vectorizer = CountVectorizer()
matrix = vectorizer.fit_transform(OMATS['cleanedSentences'])
matrix.shape

(10, 132)

With PCA, we extract the 3 most important sentences according to the weights of the sentences, and thus we have summarized the entire article.

In [15]:
projection = PCA(n_components=1)
weights = projection.fit_transform(matrix.toarray())
res = list(zip(weights.transpose()[0],range(112),OMATS['cleanedSentences']))
res

[(-0.9107803909562533,
  0,
  'rusyaukrayna savaşı devam ederken kanadanın ukraynadaki büyükelçiliğini kısa süre içinde yeniden açmayı planladığı açıklandı'),
 (-0.7991405581715022,
  1,
  'kanada dışişleri bakanı melanie joly yaptığı açıklamada hedefimiz önümüzdeki günlerde veya haftalarda bunu gerçekleştirmek'),
 (-0.7027214156335089,
  2,
  'sadece personel için güvenli bir ortam olduğundan emin olmamız gerekiyor'),
 (-0.24404909804091585,
  3,
  'ayrıca diğer meslektaşlarımızın ve müttefiklerimizin ne yaptığına bakıyoruz dedi'),
 (-1.2699458812762008,
  4,
  'kanadanın eski rusya büyükelçisi jeremy kinsman ise kanadanın diplomatik varlığını ukraynadan asla taşımaması gerektiğini belirterek asla ayrılmamalıydık'),
 (-0.5260026996917062, 5, 'büyükelçiliği kapatmamalıydık'),
 (-0.6833074111251648,
  6,
  'hükümetlerin geçmişten öğrendiklerimizi neden tekrar tekrar unuttuğunu bilmiyorum'),
 (0.0979367685181343,
  7,
  'büyükelçiliklerimiz bir hükümete akreditedir ve siz hükümetten ayrı

In [16]:
sorted(sorted(res,key=lambda x: x[0],reverse=True)[:3],key=lambda x: x[1])

[(-0.24404909804091585,
  3,
  'ayrıca diğer meslektaşlarımızın ve müttefiklerimizin ne yaptığına bakıyoruz dedi'),
 (0.0979367685181343,
  7,
  'büyükelçiliklerimiz bir hükümete akreditedir ve siz hükümetten ayrılmıyorsunuz ifadelerini kullandı kanada geçtiğimiz  şubatta ukraynanın başkenti kievdeki büyükelçiliğini kapatmış ve ukraynada rus saldırıları tehditlerinin artması üzerine diplomatik personelini ukraynanın lviv şehrine taşımıştı'),
 (5.655286955418007,
  9,
  'türkiyeden ve dünyadan son dakika haberler köşe yazıları magazinden siyasete spordan seyahate bütün konuların tek adresi milliyetcomtr milliyetcomtr haber içerikleri izin alınmadan kaynak gösterilerek dahi iktibas edilemez kanuna aykırı ve izinsiz olarak kopyalanamaz başka yerde yayınlanamaz')]

In [20]:
def getSummary(text,k):
    sentences = processText(text)
    matrix = getMatrix(sentences)
    projection = PCA(n_components=1)
    weights = projection.fit_transform(matrix.toarray())
    res = list(zip(weights.transpose()[0],range(112),OMATS['cleanedSentences']))
    tmp = sorted(res,key=lambda x: x[0],reverse=True)[:k]
    return sorted(tmp, key=lambda x: x[1])

In [21]:
getSummary(getText(url),3)

[(-0.3548798503288081,
  1,
  'kanada dışişleri bakanı melanie joly yaptığı açıklamada hedefimiz önümüzdeki günlerde veya haftalarda bunu gerçekleştirmek'),
 (1.5052188959139156e-16,
  2,
  'sadece personel için güvenli bir ortam olduğundan emin olmamız gerekiyor'),
 (0.8142642485636853,
  3,
  'ayrıca diğer meslektaşlarımızın ve müttefiklerimizin ne yaptığına bakıyoruz dedi')]

In [22]:
def getKeywords(text,sw,k):
    sentences = processText(text)
    vectorizer = CountVectorizer(stop_words=sw)
    matrix = vectorizer.fit_transform(sentences)
    words = vectorizer.get_feature_names_out()
    
    projection = PCA(n_components=1)
    tmp = projection.fit_transform(matrix.transpose().toarray())
    weights = tmp.transpose()[0]

### Q2

Write a function that returns all named entities (proper names, country names, corporation names only) from a URL. Function should take the URL as the input and must return the list of named entities from that URL. Test your code on random articles from the Guardian. Don't use the NLTK's NER that I demonstrated during the lecture. Use the SpaCY's NER function.

## Q2

In [23]:
import requests
import nltk
import regex as re
import numpy as np
import spacy

from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer

from snowballstemmer import TurkishStemmer
from bs4 import BeautifulSoup

from collections import Counter
from xmltodict import parse

In [25]:
def getSubjectGuardian(subject):
    link=requests.get(f'https://www.theguardian.com/{subject}/rss')
    raw = parse(link.text)
    return raw['rss']['channel']['item']

def getText(url):
    link=requests.get(url)
    raw = BeautifulSoup(link.content,'html.parser')
    return ' '.join([x.text for x in raw.find_all('p')])

In [29]:
def func(link):
    NER = spacy.load("en_core_web_sm")
    res = NER(getText(link))
    return [(word.text, word.label_) for word in res.ents]

### Q3

1. Write a function that returns the most positive and the most negative sentences from a text. The function must take the text as the input and must return a 2-tuple: the first element as the most positive and the second as the most negative sentence with their polarity scores.

2. Test your function on random articles from the Guardian.